# VASIM Autoscaling Simulator Toolkit Example

### Goals

1. **Applicability:** Integration with various algorithms and parameter customization.

2. **Simulation:** Realistic workload modeling, achieved within minutes.

3. **Parameter Tuning:** Fine-tuning for optimal performance and cost savings.

4. **Cost Analysis:** Demonstrating potential cost savings.


### Overview: Autoscaling Components


<img src="../docs/demo_pics/Autoscaling_infra.png" alt="Autoscaling Components" title="Autoscaling Components" width="300" >

- Application: The running software, such as Postgres and SQL Server.
- Controller: Manages tasks and publishes metrics.
- Metrics Server: Stores and provides metrics.
- Recommender Algorithm: Makes resource allocation decisions.
- Scaler Entity: Enacts decisions, adjusting resource allocation.

These components work together for effective autoscaling.


### Components

VASIM is a standalone Python package that internally replicates these components, maintaining and processing scaling states. It also includes a cluster state simulator for current utilization and limits, along with a post-processing analyzer for performance assessment. Input is in the same format as timestamp and utilization datasets. Output is a set of metrics and scaling decision trace.

VASIM allows you to try different recommender algorithms to adapt to different workloads. Workloads can be bursty, monotononic, cyclical, and business needs may vary based on complexity, cost, and performance.

<img src="../docs/demo_pics/VASim_infra.png" alt="VASIM Overview" title="VASIM Overview" width="300">




# Getting started


### Prepare data:
Get your test data. For this experiment we'll be working with the [Alibaba dataset](https://github.com/alibaba/clusterdata), trace c_12104.

There are some important things to note about the data for Vasim.

1. You currently must name your csv's ending with `perf_event_log.csv`. (Ex: `c_12104.csv_perf_event_log.csv`.)  This is so the ingester does not accidentally read in other files in the directory such as output files.
2. You must format your data in two columns (`TIMESTAMP`,`CPU_USAGE_ACTUAL`) as follows:  (TODO: We [plan](https://github.com/microsoft/vasim/issues/34) to use Open Telemetry format in the future.) 
3. You may have multiple CSVs, just put everything in the same folder.

In [1]:
! head data/c_12104.csv_perf_event_log.csv

TIMESTAMP,CPU_USAGE_ACTUAL
2023.04.02-00:09:00:000,7.2
2023.04.02-00:10:00:000,7.04
2023.04.02-00:11:00:000,6.88
2023.04.02-00:12:00:000,6.72
2023.04.02-00:13:00:000,6.48
2023.04.02-00:14:00:000,6.501818181818182
2023.04.02-00:15:00:000,6.523636363636364
2023.04.02-00:16:00:000,6.545454545454546
2023.04.02-00:17:00:000,6.567272727272727


### Prepare algorithm

You also need to implement your Recommender algorithm. There are examples in the [recommender](../recommender) folder.

To implement a recommender you will need to create a file for your algorithm, add any paramters you need to the `__init__` function and then implement the core logic of your recommender in the `run()` function.

```python
class SimpleAdditiveRecommender(Recommender):
    def __init__(self, cluster_state_provider, save_metadata=True):
        # Copy the code at the top of this function as-is.

        # Put your parameters here hard-coded, or pass them in to your
        # `metadata.json` file in the `algo_specific_config` section.
        self.my_param = self.algo_params.get("myparam", 2)

    def run(self, recorded_data):
        """
        This method runs the recommender algorithm and returns the new number of
              cores to scale to (new limit).

        Inputs:
            recorded_data (pd.DataFrame): The recorded metrics data for the current time window to simulate
        Returns:
            latest_time (datetime): The latest time of the performance data.
            new_limit (float): The new number of cores to scale to.
        """

        # Your logic goes here! Look at the data in the `recorded_data` dataframe,
        #   do a calculation, and return the number of cores to scale to.

        return new_limit
```

For this example, we'll be working with the [DummyAdditiveRecommender](../recommender/DummyAdditiveRecommender.py), which takes a moving average of the CPU values and adds a fixed buffer amount to the top.

### Preparing metadata.json
We also need to prepare a file that provides the default set of configuration parameters. You can see some examples of this file in [the test folder](../tests/test_data/alibaba_control_c_29247_denom_1_mini). The default name of this file is `metadata.json`, but you can give it any name and pass it in as a parameter.

There are three sets of parameters:
* **`algo_specific_config`** : this is where you put the parameters you will use in your algorithm's `run` function
  * An example: `addend: 2` for the `DummyAdditiveRecommender` we'll be using for this example.
* **`general_config`** : These are the parameters related to how the csv trace data is passed in, and the simulation safe-guards.
  * `window` (int): the amount of data that is passed to the algorithm in the `recorded_data` paramaeter in the `run` function. _See "original window" below_
  * `lag` (float): Number of minutes to wait after making a decision
  * `min_cpu_limit` (int): The minimum number of cores to recommend. This is used as a [safety guard](https://github.com/microsoft/vasim/blob/198a06062a91f6455b87710b0e59834530b6ea29/simulator/SimulatedInfraScaler.py#L56) in the simulated infra.
  * `max_cpu_limit` (int): The maximum number of cores to recommend. (same as min)
  * `recovery_time` (int of minutes, optional, default is 15): The **minimum** amount of time in minutes to wait before making another scaling decision.
  * `granularity` (int, unused): This needs to be removed [#35](https://github.com/microsoft/vasim/issues/35), but for now it must be an integer.
* **`prediction_config`** : this relates to the window of [predicted](https://github.com/microsoft/vasim/blob/198a06062a91f6455b87710b0e59834530b6ea29/recommender/cluster_state_provider/PredictiveFileClusterStateProvider.py#L29) data that is fed into the algorithm. It uses a time series to forecast what the data might be in the future to help the algorithm proactively scale
  * `waiting_before_predict` (int of minutes) : This is the amount of data to consume before making prediction. It is usually set to `1440`, for 60 min * 24 hours = 1 day. In general, it is waiting for one full cycle of data (`period` in the graph below) to pass. Sometimes this is daily, sometimes weekly, etc.  TODO: autodetermine this.
  * `frequency_minutes` (int) : This is how frequent your timestamps are in the csvs you provide. This *MUST* match your csvs.  (Ex: 1 in the above case.) TODO: automate.
  * `forecasting_models` (string, unused): For now we only support "naive".  This parameter is not currently used because it's the only thing supported. However, you still must provide a string for now as a placeholder
  * `minutes_to_predict` (int): This is the “forecasting horizon” below/how much to look forward.  Increasing this means you’ll be MORE proactive in adjusting based on history. This is a good one to tune depending on how consisten your data is.
  * `total_predictive_window` (int, optional): By [default](https://github.com/microsoft/vasim/blob/198a06062a91f6455b87710b0e59834530b6ea29/recommender/cluster_state_provider/PredictiveFileClusterStateProvider.py#L39), this parameter is `minutes_to_predict`/`frequency_minutes` + `window`.  If you would like to change this, you can do it here. 
   _This is the "new window" in the diagram below_, essentially the total amount of minutes you want 

Some important things to note:
* `recovery_time` means that the simulator will NOT scale up/down even if the decision indicates to scale, until that minimum number of minutes has passed. This simulates a live system recovering.
* `lag` is the number of minutes to wait between making a decision/running your `run()` function. For example, for very heavy-weight algorithms, you may only want to run every 10 minutes. This also is used to prevent thrashing.

Here is a picture that explains the `window` and `prediction_config`:

<img src="../docs/demo_pics/predictive_window.png" alt="Data Windows" title="Data Windows" >

Here is an example of the `metadata.json`:

In [2]:
! cat data/metadata.json

{
    "algo_specific_config": {
        "addend": 2
    },
    "general_config": {
        "window": 20,
        "lag": 10,
        "max_cpu_limit": 25,
        "min_cpu_limit": 2.0,
        "granularity": 1
    },
    "prediction_config": {
        "waiting_before_predict": 1440,
        "frequency_minutes": 1,
        "minutes_to_predict": 10,
        "forecasting_models": "naive"
    }
}

Once you have all three (CSV, algo, metadata), for now, there is one last step. (TODO: [Issue #19](https://github.com/microsoft/vasim/issues/19))  You must currently [add the name of your algorithm to the simulator](https://github.com/microsoft/vasim/blob/9b50d9080d4a26fbad014159484fd110af95fa75/simulator/InMemorySimulator.py#L87) for it to be called, and import it at the top of the [InMemorySimulator.py file](../simulator/InMemorySimulator.py).

```python
 def _create_recommender_algorithm(self, algorithm):
        if algorithm == 'multiplicative':
            return SimpleMultiplierRecommender(self.cluster_state_provider)
        elif algorithm == 'additive':
            return SimpleAdditiveRecommender(self.cluster_state_provider)
        # Add your own algorithm here!!!
        # TODO: Make this more dynamic
        else:
            raise ValueError(f"Unknown algorithm: {algorithm}")
```

**REMEMEBER: You must reload this notebook/restart the kernel for Python to see your new algorithm**

## Getting started with a single simulation

Now that we have the csv, algorithm, and parameter config, we can run a single simulation for a fixed set of parameters.

`initial_cpu_limit` is the current limit that your system is set to. (TODO: we could move this to `metadata.json` but we found that value changed more than the other variables in the file, so for now it is a parameter.)

In [4]:
from simulator.InMemorySimulator import InMemoryRunnerSimulator

# Specify your data directory, algorithm, initial cores count and config path if you did not name your metadata.json as metadata.json
runner = InMemoryRunnerSimulator(data_dir="data/",  algorithm="additive", initial_cpu_limit=10, config_path="data/metadata.json")

results = runner.run_simulation()

config: {'algorithm_config': {}, 'general_config': {'lag': 10, 'granularity': 1, 'min_cpu_limit': 2.0, 'max_cpu_limit': 25, 'window': 20}, 'prediction_config': {}}
logger: <RootLogger root (ERROR)>
data_dir: /home/osdi-eval/vasim/examples/data
features: ['cpu']
decision_file_path: /home/osdi-eval/vasim/examples/data/_simulations/target_79d24be0-7d45-4ae0-a53b-58fee0e01933/decisions.txt
save_metadata: True
frequency_minutes: 1
minutes_to_predict: 10
freq: 1T
waiting_time: 1440
data_processor: <recommender.forecasting.utils.DataProcessor.DataProcessor object at 0x71c52fd43ac0>
data_forecaster: <recommender.forecasting.TimeSeriesForecaster.TimeSeriesForecaster object at 0x71c52fd43280>
cores: 0
predicted_cores: 0
_prediction_activated: False
number_of_points_to_predict: 10
predictive_window: 30
curr_cpu_limit: None
JSON file successfully written.
Starting simulation at 2023-04-02 00:09:00 and continuing till 2023-04-09 23:20:00
Setting number of cores to 10
Simulation finished at 2023-04-

In [6]:
# print the
print(results)

{'average_slack': 2.6615605822802952, 'average_insufficient_cpu': 0.011724998250317914, 'sum_slack': 30477.530227691663, 'sum_insufficient_cpu': 134.26295496439045, 'num_scalings': 289, 'num_insufficient_cpu': 99, 'insufficient_observations_percentage': 0.8645533141210375, 'slack_percentage': 23.172159395780824, 'median_insufficient_cpu': 0.0, 'median_slack': 2.4399999999999995, 'max_slack': 8.236}
